In [11]:
import sys
import itertools
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler


from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from tsfresh.feature_extraction import extract_features
from multiprocessing import Pool
tqdm.pandas(desc="apply progress")

import warnings
warnings.filterwarnings('ignore')

## データ整形の関数群

In [2]:
aggs = {
    'flux': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'flux_err': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'detected': ['mean'],
    'flux_ratio_sq': ['sum', 'skew'],
    'flux_by_flux_ratio_sq': ['sum', 'skew'],
}

# agg diff features
diff_aggs = {
    'flux_diff': ['min', 'max', 'mean', 'median', 'std', 'skew'],
}

# tsfresh features
fcp = {
    'flux': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,
        'mean_change': None,
        'mean_abs_change': None,
        'length': None,
        'mean': None,
        'maximum': None,
        'minimum': None,
        # additional
        'absolute_sum_of_changes': None,
        'autocorrelation': [{'lag': 3}],
        'binned_entropy': [{'max_bins': 10}],
        'cid_ce': [{'normalize': True}],
        'count_above_mean': None,
        'first_location_of_maximum': None,
        'first_location_of_minimum': None,
        'last_location_of_maximum': None,
        'last_location_of_minimum': None,
        'mean_second_derivative_central': None,
        'median': None,
        'ratio_beyond_r_sigma': [{'r': 2}],
        'sample_entropy': None,
        'time_reversal_asymmetry_statistic': [{'lag': 4}]
    },

    'flux_by_flux_ratio_sq': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,
        # additional
        'mean_change': None,
        'mean_abs_change': None,
        'length': None,
        'mean': None,
        'maximum': None,
        'minimum': None,
        'abs_energy': None,
        'absolute_sum_of_changes': None,
        'autocorrelation': [{'lag': 3}],
        'binned_entropy': [{'max_bins': 10}],
        'cid_ce': [{'normalize': True}],
        'count_above_mean': None,
        'count_below_mean': None,
        'first_location_of_maximum': None,
        'first_location_of_minimum': None,
        'kurtosis': None,
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,
        'mean_second_derivative_central': None,
        'median': None,
        'sample_entropy': None,
        'standard_deviation': None,
        'time_reversal_asymmetry_statistic': [{'lag': 3}]
    },

    'flux_passband': {
        'fft_coefficient': [
                {'coeff': 0, 'attr': 'abs'},
                {'coeff': 1, 'attr': 'abs'}
            ],
        'kurtosis': None,
        'skewness': None,
        'maximum': None,
        'mean': None,
        'minimum': None,
        # additional
        'abs_energy': None,
        'autocorrelation': [{'lag': 3}],
        'binned_entropy': [{'max_bins': 10}],
        'cid_ce': [{'normalize': True}],
        'mean_second_derivative_central': None,
        'median': None,
        'sample_entropy': None,
        'standard_deviation': None,
        'sum_values': None,
        'time_reversal_asymmetry_statistic': [{'lag': 4}]
    },

    'fcp2':  {
        "fft_coefficient": [{
            "coeff": 0,
            "attr": "abs"
        }, {
            "coeff": 1,
            "attr": "abs"
        }],
        "abs_energy": None,
        "sample_entropy": None
    },

    'mjd': {
        'maximum': None,
        'minimum': None,
        'mean_change': None,
        'mean_abs_change': None,
    },
}

In [3]:
def haversine_plus(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees) from
    #https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    """
    # Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    # Implementing Haversine Formula:
    dlon = np.subtract(lon2, lon1)
    dlat = np.subtract(lat2, lat1)

    a = np.add(
        np.power(np.sin(np.divide(dlat, 2)), 2),
        np.multiply(
            np.cos(lat1),
            np.multiply(np.cos(lat2), np.power(np.sin(np.divide(dlon, 2)),
                                               2))))

    haversine = np.multiply(2, np.arcsin(np.sqrt(a)))
    return {
        'haversine': haversine,
        'latlon1': np.subtract(
            np.multiply(lon1, lat1), np.multiply(lon2, lat2)),
    }


def process_flux(df):
    flux_ratio_sq = np.power(df['flux'].values / df['flux_err'].values, 2.0)

    df_flux = pd.DataFrame(
        {
            'flux_ratio_sq': flux_ratio_sq,
            'flux_by_flux_ratio_sq': df['flux'].values * flux_ratio_sq,
        },
        index=df.index)

    return pd.concat([df, df_flux], axis=1)


def process_flux_agg(df):
    flux_w_mean = df['flux_by_flux_ratio_sq_sum'].values / df[
        'flux_ratio_sq_sum'].values
    flux_diff = df['flux_max'].values - df['flux_min'].values

    df_flux_agg = pd.DataFrame(
        {
            'flux_w_mean': flux_w_mean,
            'flux_diff1': flux_diff,
            'flux_diff2': flux_diff / df['flux_mean'].values,
            'flux_diff3': flux_diff / flux_w_mean,
        },
        index=df.index)

    return pd.concat([df, df_flux_agg], axis=1)


def make_diff_feature(df):
    tmp = df.groupby(['object_id', 'passband',
                      'mjd'])['flux'].sum().reset_index()

    tmp['flux_diff'] = tmp['flux'] - tmp['flux'].shift(1)
    multi_id_list = (tmp['object_id'].astype(str) + '-' +
                     tmp['passband'].astype(str)).values

    drop_index = []
    prev_val = 'hoge'
    for index, val in enumerate(multi_id_list):
        if val != prev_val:
            drop_index.append(index)
        prev_val = val

    use_index = list(set(tmp.index) - set(drop_index))
    tmp = tmp.iloc[use_index, :]
    diff_df = tmp.drop('flux', axis=1)

    return diff_df


def featurize(df, df_meta, aggs, fcp, n_jobs=72):
    """
    Extracting Features from train set
    Features from olivier's kernel
    very smart and powerful feature that is generously given here https://www.kaggle.com/c/PLAsTiCC-2018/discussion/69696#410538
    per passband features with tsfresh library. fft features added to capture periodicity https://www.kaggle.com/c/PLAsTiCC-2018/discussion/70346#415506
    """

    df = process_flux(df)

    agg_df = df.groupby('object_id').agg(aggs)
    agg_df.columns = [
        '{}_{}'.format(k, agg) for k in aggs.keys() for agg in aggs[k]
    ]
    agg_df = process_flux_agg(agg_df)  # new feature to play with tsfresh
    per_passband_aggs = {
        "flux": ["min", "max", "mean", "std"],
        "flux_ratio_sq": ["sum", "skew"],
        "flux_by_flux_ratio_sq": ["sum", "skew"]
    }
    per_pass_agg_df = df.groupby(["object_id",
                                  "passband"]).agg(per_passband_aggs)
    per_pass_agg_df.columns = pd.Index(
        [e[0] + "_" + e[1] for e in per_pass_agg_df.columns])
    per_pass_agg_df["flux_diff"] = per_pass_agg_df[
        "flux_max"] - per_pass_agg_df["flux_min"]
    per_pass_agg_df["flux_diff2"] = (
        per_pass_agg_df["flux_max"] -
        per_pass_agg_df["flux_min"]) / per_pass_agg_df["flux_mean"]
    per_pass_agg_df["flux_w_mean"] = per_pass_agg_df[
        "flux_by_flux_ratio_sq_sum"] / per_pass_agg_df["flux_ratio_sq_sum"]
    per_pass_agg_df["flux_dif3"] = (
        per_pass_agg_df["flux_max"] -
        per_pass_agg_df["flux_min"]) / per_pass_agg_df["flux_w_mean"]
    per_pass_agg_df = per_pass_agg_df.unstack()
    per_pass_agg_df.columns = pd.Index(
        [str(e[1]) + "__" + e[0] for e in per_pass_agg_df.columns])

    basic_columns = [
        f"{i}__{j}" for i in range(6) for j in [
            "flux_min", "flux_max", "flux_mean", "flux_std",
            "flux_ratio_sq_sum", "flux_ratio_sq_skew", "flux_w_mean",
            "flux_diff2"
        ]
    ]
    per_pass_agg_df.drop(basic_columns, axis=1, inplace=True)

    agg_df = pd.merge(agg_df, per_pass_agg_df, how="left", on="object_id")

    agg_flux_diff = agg_df.reset_index()[["object_id", "flux_diff1"]]
    df2 = pd.merge(df, agg_flux_diff, how="left", on="object_id")
    df2["flux_norm"] = df2.flux / df2.flux_diff1
    del df2["flux"]

    # Add more features with
    agg_df_ts_flux_passband = extract_features(
        df,
        column_id='object_id',
        column_sort='mjd',
        column_kind='passband',
        column_value='flux',
        default_fc_parameters=fcp['flux_passband'],
        n_jobs=n_jobs)

    agg_df_ts_flux = extract_features(
        df,
        column_id='object_id',
        column_value='flux',
        default_fc_parameters=fcp['flux'],
        n_jobs=n_jobs)

    agg_df_ts2 = extract_features(
        df2,
        column_id="object_id",
        column_sort="mjd",
        column_kind="passband",
        column_value="flux_norm",
        default_fc_parameters=fcp["fcp2"],
        n_jobs=n_jobs)

    agg_df_ts_flux_by_flux_ratio_sq = extract_features(
        df,
        column_id='object_id',
        column_value='flux_by_flux_ratio_sq',
        default_fc_parameters=fcp['flux_by_flux_ratio_sq'],
        n_jobs=n_jobs)

    # Add smart feature that is suggested here https://www.kaggle.com/c/PLAsTiCC-2018/discussion/69696#410538
    # dt[detected==1, mjd_diff:=max(mjd)-min(mjd), by=object_id]
    df_det = df[df['detected'] == 1].copy()
    agg_df_mjd = extract_features(
        df_det,
        column_id='object_id',
        column_value='mjd',
        default_fc_parameters=fcp['mjd'],
        n_jobs=n_jobs)
    agg_df_mjd['mjd_diff_det'] = agg_df_mjd[
        'mjd__maximum'].values - agg_df_mjd['mjd__minimum'].values
    del agg_df_mjd['mjd__maximum'], agg_df_mjd['mjd__minimum']
    agg_df_ts2.columns = pd.Index([e + "_norm" for e in agg_df_ts2.columns])
    agg_df_ts_flux_passband.index.rename('object_id', inplace=True)
    agg_df.index.rename('object_id', inplace=True)
    agg_df_ts_flux.index.rename('object_id', inplace=True)
    agg_df_ts_flux_by_flux_ratio_sq.index.rename('object_id', inplace=True)
    agg_df_mjd.index.rename('object_id', inplace=True)
    agg_df_ts = pd.concat([
        agg_df, agg_df_ts2, agg_df_ts_flux_passband, agg_df_ts_flux,
        agg_df_ts_flux_by_flux_ratio_sq, agg_df_mjd
    ],
                          axis=1).reset_index()

    result = agg_df_ts.merge(right=df_meta, how='left', on='object_id')
    return result


def diff_featurize(diff_df, df_meta, diff_aggs, fcp, n_jobs=72):
    """
    Extracting Features from train set
    Features from olivier's kernel
    very smart and powerful feature that is generously given here https://www.kaggle.com/c/PLAsTiCC-2018/discussion/69696#410538
    per passband features with tsfresh library. fft features added to capture periodicity https://www.kaggle.com/c/PLAsTiCC-2018/discussion/70346#415506
    """

    # df = train.copy()

    # diff_df = process_flux(diff_df)

    diff_agg_df = diff_df.groupby('object_id').agg(diff_aggs)
    diff_agg_df.columns = [
        '{}_{}'.format(k, agg) for k in diff_aggs.keys()
        for agg in diff_aggs[k]
    ]
    # diff_agg_df = process_flux_agg(diff_agg_df) # new feature to play with tsfresh

    # Add more features with
    diff_agg_df_ts_flux_passband = extract_features(
        diff_df,
        column_id='object_id',
        column_sort='mjd',
        column_kind='passband',
        column_value='flux_diff',
        default_fc_parameters=fcp['flux_passband'],
        n_jobs=n_jobs)

    diff_agg_df_ts_flux = extract_features(
        diff_df,
        column_id='object_id',
        column_value='flux_diff',
        default_fc_parameters=fcp['flux'],
        n_jobs=n_jobs)

    diff_agg_df_ts_flux_passband.index.rename('object_id', inplace=True)
    diff_agg_df_ts_flux_passband.columns = [
        column + '_diff' for column in diff_agg_df_ts_flux_passband.columns
    ]
    diff_agg_df_ts_flux.index.rename('object_id', inplace=True)
    # agg_df_ts_flux_by_flux_ratio_sq.index.rename('object_id', inplace=True)
    # agg_df_mjd.index.rename('object_id', inplace=True)
    diff_agg_df_ts = pd.concat(
        [
            diff_agg_df,
            diff_agg_df_ts_flux_passband,
            diff_agg_df_ts_flux,
            # agg_df_ts_flux_by_flux_ratio_sq,
            # agg_df_mjd
        ],
        axis=1).reset_index()

    # result = agg_df_ts.merge(right=df_meta, how='left', on='object_id')
    result = diff_agg_df_ts
    return result


def process_meta(meta_df):
    meta_dict = dict()

    # id trick
    for i in [22, 27]:
        meta_dict['object_id_div_{}'.format(i)] = np.mod(
            meta_df['object_id'].values, i)

    # distance
    meta_dict.update(
        haversine_plus(meta_df['ra'].values, meta_df['decl'].values,
                       meta_df['gal_l'].values, meta_df['gal_b'].values))
    #
    meta_dict['hostgal_photoz_certain'] = np.multiply(
        meta_df['hostgal_photoz'].values,
        np.exp(meta_df['hostgal_photoz_err'].values))

    meta_df = pd.concat(
        [meta_df, pd.DataFrame(meta_dict, index=meta_df.index)], axis=1)
    return meta_df


def add_rank_bottom_and_top(df, feature_name):
    objid = ["object_id"]
    columns = [f"{i}{feature_name}" for i in range(6)]
    partial_df = df[objid + columns]
    partial_values = partial_df.melt(
        id_vars=objid, value_vars=columns).sort_values(["object_id", "value"])

    top_and_bottom = partial_values.groupby("object_id").agg({
        "variable": ["first", "last"]
    })
    top_and_bottom.columns = ["top" + feature_name, "bottom" + feature_name]
    for i, n in zip(["0", "1", "2", "3", "4", "5"], columns):
        top_and_bottom = top_and_bottom.replace(n, i)
    top_and_bottom = top_and_bottom.astype(int)
    return top_and_bottom


def add_by_features(df, feature_name, new_feat_name):
    for i in range(5):
        for j in range(1, 6):
            if j > i:
                df[f"{new_feat_name}{j}_by_{i}"] = df[
                    f"{j}{feature_name}"] / df[f"{i}{feature_name}"]
    return df

## 予測関連の関数群

In [4]:
def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {
        6: 1,
        15: 2,
        16: 1,
        42: 1,
        52: 1,
        53: 1,
        62: 1,
        64: 2,
        65: 1,
        67: 1,
        88: 1,
        90: 1,
        92: 1,
        95: 1
    }
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array(
        [class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = -np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {
        6: 1,
        15: 2,
        16: 1,
        42: 1,
        52: 1,
        53: 1,
        62: 1,
        64: 2,
        65: 1,
        67: 1,
        88: 1,
        90: 1,
        92: 1,
        95: 1
    }
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array(
        [class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = -np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

def multi_weighted_logloss_novae(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [15, 42, 52, 62, 67, 90]
    class_weight = {
        15: 2,
        42: 1,
        52: 1,
        62: 1,
        67: 1,
        90: 1,
    }
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array(
        [class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = -np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss_novae(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [15, 42, 52, 62, 67, 90]
    class_weight = {
        15: 2,
        42: 1,
        52: 1,
        62: 1,
        67: 1,
        90: 1,
    }
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array(
        [class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = -np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False


def multi_weighted_logloss_nonnovae(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 16, 53, 64, 65, 88, 92, 95]
    class_weight = {6: 1, 16: 1, 53: 1, 64: 2, 65: 1, 88: 1, 92: 1, 95: 1}
    
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array(
        [class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = -np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss_nonnovae(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 16, 53, 64, 65, 88, 92, 95]
    class_weight = {6: 1, 16: 1, 53: 1, 64: 2, 65: 1, 88: 1, 92: 1, 95: 1}
    
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array(
        [class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = -np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [12]:
# single_superの特徴量整形用関数
def train_data_super(train, meta_train):
    y = meta_train.target
    meta_train = process_meta(meta_train)
    
    # 特徴量作成
    full_train = featurize(train, meta_train, aggs, fcp, n_jobs=4)
    # print('full train shape :', full_train.shape)

    # 差分特徴量（ver0で作成）
    diff_train = make_diff_feature(train)
    diff_full_train = diff_featurize(diff_train, meta_train, diff_aggs, fcp, n_jobs=4)

    # 特徴量のマージ
    full_train = pd.merge(full_train, diff_full_train, on='object_id')

    # ランクの特徴量を追加
    full_train = add_by_features(
        full_train, "__fft_coefficient__coeff_0__attr_\"abs\"_norm",
        "flux_norm_fft_")
    full_train = add_by_features(full_train, "__abs_energy_norm", "abs_energy_")
    full_train = add_by_features(full_train, "__flux_diff", "flux_diff_")
    abs_energy = add_rank_bottom_and_top(full_train, "__abs_energy_norm")
    flux_diff = add_rank_bottom_and_top(full_train, "__flux_diff")
    flux_dif3 = add_rank_bottom_and_top(full_train, "__flux_dif3")

    full_train = pd.merge(full_train, abs_energy, how="left", on="object_id")
    full_train = pd.merge(full_train, flux_diff, how="left", on="object_id")
    full_train = pd.merge(full_train, flux_dif3, how="left", on="object_id")
    
    if 'target' in full_train:
        # y = full_train['target']
        del full_train['target']

    classes = sorted(y.unique())
    
    class_weights = {c: 1 for c in classes}
    class_weights.update({c: 2 for c in [64, 15]})
    
    oof_df = full_train[["object_id"]]
    
    # モデル前整形
    if 'object_id' in full_train:
        del full_train['object_id']
        # del full_train['distmod']
        del full_train['hostgal_specz']
        del full_train['ra'], full_train['decl'], full_train['gal_l'], full_train['gal_b']
        del full_train['ddf']
    
    full_train = full_train.replace(np.inf, np.nan)
    full_train = full_train.replace(-np.inf, np.nan)

    scl = StandardScaler()
    full_train = pd.DataFrame(scl.fit_transform(full_train), index=full_train.index, columns=full_train.columns)
    full_train.fillna(0, inplace=True)
    
    use_features = pd.read_csv('./use_features_5-0.csv')['use_feature'].tolist()
    full_train = full_train[use_features]
    
    return full_train, y, classes, class_weights, oof_df, scl

In [5]:
def save_importances(importances_):
    mean_gain = importances_[['gain', 'feature']].groupby('feature').mean()
    importances_['mean_gain'] = importances_['feature'].map(mean_gain['gain'])
    plt.figure(figsize=(8, 12))
    sns.barplot(
        x='gain',
        y='feature',
        data=importances_.sort_values('mean_gain', ascending=False)[:300])
    plt.tight_layout()
    plt.savefig('importances_novae.png')


def plot_confusion_matrix(cm,
                          classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


def save_cm(y, oof_preds):
    unique_y = np.unique(y)
    class_map = dict()
    for i, val in enumerate(unique_y):
        class_map[val] = i

    y_map = np.zeros((y.shape[0], ))
    y_map = np.array([class_map[val] for val in y])

    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_map, np.argmax(oof_preds, axis=-1))
    np.set_printoptions(precision=2)

    class_names = ["class_15", "class_42", "class_52", "class_62", "class_67", "class_90"]
    # Plot non-normalized confusion matrix
    plt.figure(figsize=(12, 12))
    plot_confusion_matrix(
        cnf_matrix,
        classes=class_names,
        normalize=True,
        title='Confusion matrix')
    plt.savefig("confusion_matrix_novae.png")
    
def plot_importances(clfs, features):
    importances = []
    for clf in clfs:
        importances.append(clf.feature_importances_)
    importances = pd.Series(np.array(importances).mean(axis=0), index=features)
    importances = importances.sort_values(ascending=False)

    fig, ax = plt.subplots(1,1,figsize=(14,14))
    importances.iloc[:40].plot(kind='barh', ax=ax)
    fig.tight_layout()

    return importances

In [6]:
def smoteAdataset(Xig_train, yig_train, Xig_test, yig_test):     
    sm=SMOTE(random_state=2)
    Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())
    return Xig_train_res, pd.Series(yig_train_res), Xig_test, pd.Series(yig_test)

In [7]:
def lgbm_modeling_cross_validation(params,
                                   full_train, 
                                   y, 
                                   classes, 
                                   class_weights,
                                   lgb_metric,
                                   metric,
                                   nr_fold=12, 
                                   random_state=1, 
                                   save_plot=True, 
                                   show_log=True,):

    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}
   # print(weights)
   # weights=class_weights
    clfs = []
    importances = pd.DataFrame()
    folds = StratifiedKFold(n_splits=nr_fold, 
                            shuffle=True, 
                            random_state=random_state)
    
    oof_preds = np.zeros((len(full_train), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = full_train.iloc[trn_], y.iloc[trn_]
        val_x, val_y = full_train.iloc[val_], y.iloc[val_]
        
                
        trn_xa, trn_y, val_xa, val_y=smoteAdataset(trn_x.values, trn_y.values, val_x.values, val_y.values)
        trn_x=pd.DataFrame(data=trn_xa, columns=trn_x.columns)
    
        val_x=pd.DataFrame(data=val_xa, columns=val_x.columns)
        
        categorical_feature = []
        categorical_feature = list(set(categorical_feature) & set(full_train.columns))
        
        clf = lgb.LGBMClassifier(**params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgb_metric,
            verbose=100,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights),
            categorical_feature=categorical_feature
        )
        clfs.append(clf)

        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        
        if show_log:
            print('no {}-fold loss: {}'.format(fold_ + 1, 
                  metric(val_y, oof_preds[val_, :])))
    
        imp_df = pd.DataFrame({
                'feature': full_train.columns,
                'gain': clf.feature_importances_,
                'fold': [fold_ + 1] * len(full_train.columns),
                })
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

    score = metric(y_true=y, y_preds=oof_preds)
    print('MULTI WEIGHTED LOG LOSS: {:.5f}'.format(score))
    
    if save_plot:
        df_importances = save_importances(importances_=importances)
    
    return clfs, score, oof_preds

## データ読み込んだり諸々

In [9]:
# data_dir = "/home/hidehisa/.kaggle/competitions/plasticc"
data_dir = "/Users/hidehisa/.kaggle/competitions/plasticc"
# train = pd.read_csv(data_dir + "/train_with_cluster.csv")
train = pd.read_csv(data_dir + "/training_set.csv")
meta = pd.read_csv(data_dir + "/training_set_metadata.csv")

In [10]:
nova = [15, 42, 52, 62, 67, 90]
nonnova = [6, 16, 53, 64, 65, 88, 92, 95]
novaes = meta.query("target == @nova")
nonnovaes = meta.query("target == @nonnova")
object_ids_novae = novaes['object_id'].tolist()
object_ids_nonnovae = nonnovaes['object_id'].tolist()

In [13]:
full, y, classes, class_weight, oof_df, scl = train_data_super(train, meta)

Feature Extraction: 100%|██████████| 20/20 [02:08<00:00,  5.58s/it]


FileNotFoundError: File b'./use_features_5-0.csv' does not exist

In [ ]:
full_train, full_test, y_train, y_test = train_test_split(full, y, test_size=0.1, random_state=7)

In [ ]:
index_novae = oof_df[oof_df['object_id'].isin(object_ids_novae)].index
index_nonnovae = oof_df[oof_df['object_id'].isin(object_ids_nonnovae)].index

In [ ]:
# 二値分類用のカラムを追加
train_bin = train.copy()
train_bin["novae"] = 0
ind = train_bin.query("object_id in @novaes.object_id").index
train_bin.loc[ind, "novae"] = 1